#### 제주도 핫플레이스 인스타그램에서 크롤링하기

##### matplotlib 한글설정

##### 셀레니움 설정

In [1]:
import matplotlib
from matplotlib import pyplot as plt
# import matplotlib.pyplot as plt
from matplotlib import font_manager, rc, rcParams
import seaborn as sns

font_path = 'C:/Windows/Fonts/malgun.ttf'
font = font_manager.FontProperties(fname=font_path).get_name() # malgun
rc('font', family=font)
rcParams['axes.unicode_minus'] = False

In [4]:
# 셀레니움
# 모듈 임포트
from selenium import webdriver

driver = webdriver.Chrome()
url = 'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91/'
driver.get(url)

##### 첫번째 게시글 찾아서 클릭하기

In [11]:
from selenium.webdriver.common.by import By
import time

def select_first_post(driver):
    first = driver.find_elements(By.CLASS_NAME, '_aagw')[0]
    first.click()
    time.sleep(3)

select_first_post(driver)

##### 게시글 정보 가져오기

In [48]:
import re # RegularExpression 정규식 처리 모듈
from bs4 import BeautifulSoup
import unicodedata # 유니코드 데이터 처리 모듈

def get_content(driver):
    # 1. 데이터 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 2. 본문내용 찾아오기
    try:
        content = soup.select('div._a9zr')[0].text
    except:
        content = ''

    # 3. 본문내용에서 해시태그 가져오기(정규식)
    tags = re.findall(r'#[^\s#,\\]+', content)

    # 4. 작성일자 정보
    data = soup.select('time._aaqe')[0]['datetime'][:10]

    # 5. 좋아요수 가져오기
    try:
        like = int(soup.select('span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp > span')[0].text)
    except:
        like = 0

    # 6. 위치정보
    # 정규식 학습필수
    try:
        regex = r'(\w+[시,읍]\s*)'\
                r'?(\w+\d*\w*[로,리,길]\s*)'\
                r'?(\w*\d+-?\d*)?'\

        place = re.findall(regex, content)
        for pl in place:
            if pl[0] != '' and pl[1] != '' and pl[2] != '':
                address = f'{pl[0]} {pl[1]} {pl[2]}'

    except:
        address = ''

    # 7. 정보 저장
    data = [content, data, like, address, tags]
    return data

get_content(driver)

['y1n_hqy(광고)제주도에서 쓰끼다시가 가장 잘나오고 서귀포에서 오션뷰로 회를 먹을수있는곳💜💜대포항 바로 앞에 큰 규모의 횟집에서 오션뷰로 즐길 수 있는 횟집으로,제주에서 횟감 하나만큼은 가장 싱싱하다고 자부할 수 있는 이곳!!제주도에서 스끼다시가 가장 잘 나오는 곳으로그때그때 가장 싱싱한 해산물만을 즐길 수 있는 맛집커플, 가족단위로 와서 힐링하기도 좋고대규모 좌석으로 대형버스 예약까지 많은 맛집!!👉🏻식사 후에는 바로 옆에 dp159카페에서 커피한잔까지 완벽한 맛집☕️☕️OCEAN VIEW 중문 큰갯물횟집0507-1360-1625제주 서귀포시 대포로 16111:50 - 22:0021:00 라스트오더#서귀포맛집 #제주중문맛집#서귀포횟집#중문횟집#제주도횟집#중문맛집#제주횟집#제주도민맛집#제주오션뷰맛집#서귀포맛집추천#제주도맛집8시간',
 '2023-06-28',
 0,
 '서귀포시  대포로  16111',
 ['#서귀포맛집',
  '#제주중문맛집',
  '#서귀포횟집',
  '#중문횟집',
  '#제주도횟집',
  '#중문맛집',
  '#제주횟집',
  '#제주도민맛집',
  '#제주오션뷰맛집',
  '#서귀포맛집추천',
  '#제주도맛집8시간']]

##### 다음 게시글 열기

In [49]:
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, ('._aaqg._aaqh'))
    right.find_element(By.XPATH, '//button')
    right.click()
    time.sleep(3)

move_next(driver)

#### 자동 크롤링 구현

In [82]:
# 모듈 임포트 생략

driver = webdriver.Chrome()
url = 'https://www.instagram.com'
driver.get(url)
time.sleep(5)
# select_first_post(driver)
# get_content(driver)
# move_next(driver)

In [83]:
def insta_login():
    id = '01092038196'
    pw = '비밀번호'
    input_id = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[0]
    input_id.clear()
    input_id.send_keys(id)
    input_pw = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[1]
    input_pw.clear()
    input_pw.send_keys(pw)
    input_pw.submit()
    time.sleep(2.0)
    
insta_login()

In [84]:
# 알림설정 버튼클릭
def btn_click():
    alarm_btn = driver.find_element(By.CSS_SELECTOR, '._a9--._a9_0')
    alarm_btn.click()
btn_click()

In [66]:
# 제주도 맛집 찾기 함수
def insta_searching(word):
    url = f'https://www.instagram.com/explore/tags/{word}'
    return url

In [87]:
# 크롤링 시작하기
driver = webdriver.Chrome()
url = 'https://www.instagram.com'
driver.get(url)
time.sleep(5)
insta_login()
btn_click()
word = '제주도맛집'
url = insta_searching(word)
driver.get(url)
time.sleep(10.0)

select_first_post(driver)

result = []

target = 5
for i in range(target):
    try:
        data = get_content(driver)
        result.append(data)
        move_next(driver)
    except:
        time.sleep(2.0)
        move_next(driver)

print(result[:5])

[['me_vonep(광고)제주도에서 쓰끼다시가 가장 잘나오고 서귀포에서 오션뷰로 회를 먹을수있는곳💜💜대포항 바로 앞에 큰 규모의 횟집에서 오션뷰로 즐길 수 있는 횟집으로,제주에서 횟감 하나만큼은 가장 싱싱하다고 자부할 수 있는 이곳!!제주도에서 스끼다시가 가장 잘 나오는 곳으로그때그때 가장 싱싱한 해산물만을 즐길 수 있는 맛집커플, 가족단위로 와서 힐링하기도 좋고대규모 좌석으로 대형버스 예약까지 많은 맛집!!👉🏻식사 후에는 바로 옆에 dp159카페에서 커피한잔까지 완벽한 맛집☕️☕️OCEAN VIEW 중문 큰갯물횟집0507-1360-1625제주 서귀포시 대포로 16111:50 - 22:0021:00 라스트오더#서귀포맛집 #제주중문맛집#서귀포횟집#중문횟집#제주도횟집#중문맛집#제주횟집#제주도민맛집#제주오션뷰맛집#서귀포맛집추천#제주도맛집1시간', '2023-06-29', 0, '서귀포시  대포로  16111', ['#서귀포맛집', '#제주중문맛집', '#서귀포횟집', '#중문횟집', '#제주도횟집', '#중문맛집', '#제주횟집', '#제주도민맛집', '#제주오션뷰맛집', '#서귀포맛집추천', '#제주도맛집1시간']], ['beli._azagi._u(광고)애월에서 가성비 좋은 흑돼지 전문점을 찾는다면 "애월그때그집"제주 그때그집의 애월본점 ! 💗 제주에서 12곳이나 지점이 있는 규모있는 흑돼지 전문점으로 한번 방문하면 또 생각나서 방문하는 애월그때그집"제주 애월맛집에 가성비 좋은 흑돼지 전문점! 두툼한 흑돼지 오겹살이 일품으로 흑돼지 본연의 고소한 맛을 즐길수있는맛집같이 나오는 반찬들은 고기와는 다 잘어울리는데 그중❗❗ 김치전골은 꼭!!! 흑돼지와 함께 먹어야하는 찐조합❤❤❤❤👉🏻제주에 12곳이나 지점이있는 흑돼지 전문점으로서, 제주 어디서나 흑돼지가 생각나면 "그때그집"애월그때그집 본점064-799-9229제주 제주시 애월읍 애월해안로 97매일 11:00 - 22:00 (화요일 휴무)#제주애월맛집#애월맛집#제주도흑돼지맛집#제주흑돼지맛집#애월흑돼지#제주흑돼지#애월